In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras.metrics as metrics
from keras.layers import Dense, LSTM, Bidirectional, GRU, Conv1D
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
cols = {
    
    # 'ts': int,
    # 'src_ip': 'category',
    'src_port': float,
    # 'dst_ip': 'category',
    'dst_port': float,
    'proto': 'category',
    # 'service': 'category',
    # 'duration': float,
    # 'src_bytes': float,
    # 'dst_bytes': float,
    # 'conn_state': 'category',
    # 'missed_bytes': float,

    # 'src_pkts': float,
    # 'src_ip_bytes': float,
    # 'dst_pkts': float,
    # 'dst_ip_bytes': float,

    # 'dns_query': 'category',
    # 'dns_qclass': float,
    # 'dns_qtype': float,
    # 'dns_rcode': float,
    # 'dns_AA': float, #bool
    # 'dns_RD': float, #bool
    # 'dns_rejected': float, #bool

    # 'ssl_version': 'category',
    # 'ssl_cipher': 'category',
    # 'ssl_resumed': float, #bool
    # 'ssl_established': float, #bool
    # 'ssl_subject': 'category',
    # 'ssl_issuer': 'category',

    # 'http_trans_depth': float,
    # 'http_method': 'category',
    # 'http_uri': 'category',
    # 'http_version': 'category',
    # 'http_request_body_len': float,
    # 'http_response_body_len': float,
    # 'http_status_code': float,
    # 'http_user_agent': 'category',
    # 'http_orig_mime_types': 'category',
    # 'http_resp_mime_types': 'category',

    # 'weird_name': 'category',
    # 'weird_addl': 'category',
    # 'weird_notice': float, #bool

    'label': int

}

def bool_to_number(x: str) -> float:
  if x == 'T':
    return 1
  elif x == 'F':
    return 0
  elif x == '-':
    return 0.5
  else:
    raise x

converters = {
    'dns_AA': bool_to_number,
    'dns_RD': bool_to_number,
    'dns_rejected': bool_to_number,
    'ssl_resumed': bool_to_number,
    'ssl_established': bool_to_number,
    'weird_notice': bool_to_number
}

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/datasci/ton_iot/Train_Test_Network.csv',
                      sep=',',
                      header=0,
                      dtype=cols,
                      usecols=list(cols.keys()),
                      converters=converters,
                      na_values=['-'])

In [ ]:
df

,src_port,dst_port,proto,label
0,1883.0,52976.0,tcp,0
1,47260.0,15600.0,udp,0
2,1880.0,51782.0,tcp,0
3,34296.0,10502.0,tcp,0
4,46608.0,53.0,udp,0
...,...,...,...,...
461038,33108.0,80.0,tcp,1
461039,37242.0,443.0,tcp,1
461040,4444.0,49178.0,tcp,1
461041,60816.0,443.0,tcp,1


In [ ]:
features = list(df.columns)
features.remove('label')

In [ ]:
for col,typ in dict(df.dtypes).items():
  if typ == 'category':
    df[col] = df[col].cat.codes.astype(int)
  else:
    vs = list(df[col].unique())
    # should not happen in the full dataset, otherwise would be removed in 1st step of preproc
    if (len(vs) == 1) and (np.isnan(vs[0])):
      print('error',col,'const column with NaN. Replacing with 0')
      df[col] = 0
    elif (len(vs) == 2) and (np.isnan(vs[0]) or np.isnan(vs[1])):
      val = vs[1] if np.isnan(vs[0]) else vs[0]
      val = -val if val != 0 else 1
      df[col] = df[col].fillna(value=val)
    else:
      df[col] = df[col].fillna(df[col].mean())

In [ ]:
X = df.drop('label',axis=1).values
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
wsize = 100
I = np.zeros((len(X), wsize, len(X[0])))
for i in range(len(X)):
  for j in range(wsize):
    if i >= j:
      I[i][j] = X[i-j]


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(I, 
                                                    df['label'].values, 
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
model = tf.keras.models.Sequential()

model.add(Bidirectional(LSTM(64, activation='tanh', kernel_regularizer='l2')))
model.add(Dense(128, activation = 'relu', kernel_regularizer='l2'))
model.add(Dense(1, activation = 'sigmoid', kernel_regularizer='l2'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy', metrics.Precision(), metrics.Recall(), metrics.AUC() ])

In [ ]:
history = model.fit(X_train,
                    Y_train,
                    epochs=20,
                    verbose = 1)

Epoch 1/20
10086/10086 [==============================] - 712s 70ms/step - loss: 0.1890 - accuracy: 0.9591 - precision: 0.9264 - recall: 0.9590 - auc: 0.9844
Epoch 2/20
10086/10086 [==============================] - 720s 71ms/step - loss: 0.1087 - accuracy: 0.9798 - precision: 0.9525 - recall: 0.9914 - auc: 0.9929
Epoch 3/20
10086/10086 [==============================] - 712s 71ms/step - loss: 0.1071 - accuracy: 0.9802 - precision: 0.9537 - recall: 0.9915 - auc: 0.9931
Epoch 4/20
10086/10086 [==============================] - 712s 71ms/step - loss: 0.1004 - accuracy: 0.9815 - precision: 0.9563 - recall: 0.9925 - auc: 0.9945
Epoch 5/20
10086/10086 [==============================] - 707s 70ms/step - loss: 0.0933 - accuracy: 0.9816 - precision: 0.9566 - recall: 0.9923 - auc: 0.9960
Epoch 6/20
10086/10086 [==============================] - 705s 70ms/step - loss: 0.1087 - accuracy: 0.9790 - precision: 0.9513 - recall: 0.9905 - auc: 0.9935
Epoch 7/20
10086/10086 [============================

In [ ]:
predict = model.predict(X_test, verbose=1)

4323/4323 [==============================] - 77s 18ms/step


In [ ]:
tp,tn,fp,fn = 0,0,0,0
predictn = predict.flatten().round().tolist()
len(predictn)
for i in range(len(predictn)):
  if predictn[i]==1 and Y_test[i]==1:
    tp+=1
  elif predictn[i]==0 and Y_test[i]==0:
    tn+=1
  elif predictn[i]==0 and Y_test[i]==1:
    fp+=1
  elif predictn[i]==1 and Y_test[i]==0:
    fn+=1
    
print(tp,tn,fp,fn)
classification_report(Y_test, predictn, output_dict=True)

47190 88760 1151 1212


{'0': {'f1-score': 0.9868636836165731,
  'precision': 0.9871984518023379,
  'recall': 0.9865291423998578,
  'support': 89972},
 '1': {'f1-score': 0.9755744601676607,
  'precision': 0.9749597124085781,
  'recall': 0.9761899836577647,
  'support': 48341},
 'accuracy': 0.9829155610824724,
 'macro avg': {'f1-score': 0.9812190718921169,
  'precision': 0.981079082105458,
  'recall': 0.9813595630288112,
  'support': 138313},
 'weighted avg': {'f1-score': 0.9829180505181379,
  'precision': 0.9829209587175682,
  'recall': 0.9829155610824724,
  'support': 138313}}

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 128)              34816     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 51,457
Trainable params: 51,457
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.save('/content/drive/Shareddrives/datasci/ton_iot/LSTM.tf')
with open('/content/drive/Shareddrives/datasci/ton_iot/result/LSTM.csv','w') as infile:
  txt = ''
  for pred in predictn:
    txt += str(int(pred))+','
  txt = txt[:-1]
  txt += '\n'
  infile.write(txt)

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/datasci/ton_iot/LSTM.tf/assets


INFO:tensorflow:Assets written to: /content/drive/Shareddrives/datasci/ton_iot/LSTM.tf/assets
